In [1]:
def linear_kernel(x,y):
    return 1.*np.dot(x.T,y)
    

class KernelSVM(BaseEstimator):
    def __init__(self, C, kernel=linear_kernel):
        self.C = C
        self.kernel = kernel
        
    def fit(self, X, Y):
        """Fit Kernel SVM using training dataset (X, Y).
        
        :param X: data samples of shape (N, d).
        :param Y: data target labels of size (N). Each label is either 1 or -1. Denoted as t_i in Bishop.
        """
        N = len(Y)
        Y = Y.reshape((-1,1)) # reshape Y 
        
        K = np.zeros((N,N))
        for i in range(N):
            for j in range(N):
                K[i,j] = self.kernel(X[i], X[j])
                
        #------------------------------------------------------------------------------------------
        # Construct appropriate matrices here to solve the optimization problem described above.
        P = matrix(Y * Y.T * K)
        q = -1.* matrix(np.ones(N))
        G = matrix(np.bmat([[-1.*np.identity(N)],[1.*np.identity(N)]]))
        h = matrix(np.bmat([[np.zeros((N,1))],[1.*self.C*np.ones((N,1))]]))
        A = matrix(1.* Y.reshape((1,-1)))
        b = matrix(np.zeros(1))
        
        #------------------------------------------------------------------------------------------

        sol = solvers.qp(P, q, G, h, A, b)
        self.alpha_ = np.array(sol['x']).flatten()
        
        #------------------------------------------------------------------------------------------
        # Find support vectors. Must be a boolean array of length N having True for support
        # vectors and False for the rest.
        self.support_vectors = self.alpha_ > 1e-5
        #------------------------------------------------------------------------------------------
        
        sv_ind = self.support_vectors.nonzero()[0]     
        self.X_sup = X[sv_ind]
        self.Y_sup = Y[sv_ind]
        self.alpha_sup = self.alpha_[sv_ind]
        self.n_sv = len(sv_ind)     
        
        #------------------------------------------------------------------------------------------
        # Compute bias
        
        sum_ = 0
        for i in range(self.n_sv):
            sum_ += Y[i]
            for j in range(self.n_sv):
                k_sv = self.kernel(self.X_sup[i],self.X_sup[j])
                sum_ = sum_ -  k_sv*self.alpha_sup[j]*self.Y_sup[j] 
                            
        self.bias_ = 1/self.n_sv * sum_
        print("bias",self.bias_)
        
        #------------------------------------------------------------------------------------------
        
    def predict_proba(self, X):
        """
        Make real-valued prediction for some new data.
        :param X: data samples of shape (N, d).
        :return: an array of N predicted scores.
        """
        alpha_sup = self.alpha_sup
        X_sup = self.X_sup
        Y_sup = self.Y_sup
        n_sv = len(X_sup)
        N = X.shape[0]
        
        print("Num sup vec.",n_sv)
        y_hat = np.zeros(N)
        
        for i in range(N):
            sum_ = 0
            data_point = X[i]
            for j in range(n_sv): 
                kernel_val = self.kernel(X_sup[j],data_point)
                sum_ += alpha_sup[j]*Y_sup[j]*kernel_val
                
            y_hat[i] =  sum_ + self.bias_
            
        return y_hat.flatten()

    def predict(self, X):
        """
        Make binary prediction for some new data.
        :param X: data samples of shape (N, d).
        :return: an array of N binary predicted labels from {-1, 1}.
        """
        return np.sign(self.predict_proba(X))
    

NameError: name 'BaseEstimator' is not defined